<a href="https://colab.research.google.com/github/jithinmjoison/Fastai_app/blob/main/Implementation_Backpropagation_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)



In [3]:
from pathlib import Path
import pickle, gzip, math, os, time, shutil, matplotlib as mpl, matplotlib.pyplot as plt

In [4]:

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

In [5]:
from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [6]:
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [7]:
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

Foundations Version 

In [8]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [9]:
# Hidden Layers for the neural network 
nh = 50

In [11]:
# Defining the weights and Biases

w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [12]:
# Defining the linear function for the linear layer

def lin(x,w,b):
  return x@w+b

In [14]:
# testing the function to check the shape it returns

t = lin(x_valid,w1,b1)
t.shape

torch.Size([10000, 50])

In [15]:
# Defining the RELU funtion 
def relu(x):
  return x.clamp_min(0.)

In [16]:
# Testing the RELU function 

t = relu(t)
t

tensor([[ 5.32,  0.00,  0.00,  ...,  5.52,  1.60,  0.00],
        [ 4.60,  0.00,  0.00,  ..., 11.82,  0.00, 10.16],
        [ 0.00,  0.00,  6.69,  ...,  1.20,  0.00,  0.00],
        ...,
        [10.28,  0.00,  1.61,  ...,  0.00,  0.00,  0.09],
        [ 4.28,  0.00,  8.05,  ...,  0.00,  0.00,  0.00],
        [ 8.67,  0.00,  0.00,  ...,  0.00,  1.46,  3.79]])

In [17]:
# Definig the neural network model and the layers associated with it 

def model(xb):
  l1 = lin(xb,w1,b1)
  l2 = relu(l1)
  return lin (l2,w2,b2)





In [25]:
#Checking the model function and implementing it 
res = model(x_valid)
res.shape

torch.Size([10000, 1])

In [19]:
#Loss function MSE

res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [21]:
res[:,0].shape , res.squeeze().shape

(torch.Size([10000]), torch.Size([10000]))

In [24]:
(res[:,0]- y_valid).shape

torch.Size([10000])

In [26]:
# Mapping y train and y_valid
y_train, y_valid = y_train.float(),y_valid.float()

preds = model(x_train)

preds.shape

torch.Size([50000, 1])

In [27]:
# Defining a silly Loss function

def mse(output, target):
  return (output[:,0] - target).pow(2).mean()

In [28]:
#testing the MSE
mse(preds,y_train)

tensor(1676.63)

In [33]:
#Gradients and backward pass. We are manually defining the Gradient for x^2

def lin_grad(inp, out, w, b):
# gradient of matrixmultiplication with respect to input 
  inp.g = out.g @ w.t()
  w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
  b.g = out.g.sum(0)


In [34]:
def forward_and_backward(inp,targ):
  # forward pass
  l1 = lin(inp,w1, b1)
  l2 = relu(l1)
  out = lin(l2,w2,b2)
  diff = out[:,0]- targ
  loss = diff.pow(2).mean()

  #backward pass:

  out.g = 2. * diff[:,None]/inp.shape[0]
  lin_grad(l2, out,w2, b2)
  l1.g = (l1>0).float() * l2.g
  lin_grad(inp, l1, w1,b1)


In [36]:
#Testing backward and forward with x_train and y_train
forward_and_backward(x_train, y_train)


In [54]:
# Refactor Model Layers as classes

class Relu():
    def __call__(self, inp):
      self.inp = inp
      self.out = inp.clamp_min(0.)
      return self.out

    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g


In [55]:
class Lin():
    def __init__(self,w,b):
        self.w = w
        self.b = b
    
    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

  


In [56]:
class Mse():
      def __call__(self, inp , targ):
          self.inp = inp
          self.targ = targ
          self.out = mse(inp , targ)
          return self.out

      def backward(self):
           self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]
 


    



In [57]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [58]:
model = Model(w1, b1, w2, b2)

In [59]:
loss = model(x_train, y_train)

In [60]:
model.backward()